In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
import gc
from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
train = pd.read_csv('./data/train/ord_train.csv')
testa = pd.read_csv('./data/test/ord_testA.csv', encoding = 'GB2312')
testb = pd.read_csv('./data/test/ord_testB.csv', encoding = 'GB2312')

train.shape
train.head()
testa.shape
testb.shape
testa.head()

(4023086, 25)

,orderid,orderdate,city,countryid,hotel,zone,room,isholdroom,arrival,etd,ordadvanceday,confirmdate,orderstatus,price,commission,noroom,masterbasicroomid,masterhotelid,supplierid,isvendor,hotelbelongto,isebookinghtl,hotelstar,supplierchannel,ordroomnum
0,3678052984,2017-05-15 10:03:34,1777,111,6459206,3302.0,62786796,F,2017-06-23,2017-06-25,40,NaN,C,717.6,57.60000,0,21087625.0,713478,1897.0,0,SHT,T,5,合作,1
1,3623712471,2017-05-02 21:25:37,622,4,4515821,NaN,51183791,F,2017-05-05,2017-05-06,3,2017-05-02 21:25:45,S,99.0,7.98600,0,30110898.0,1210465,2322.0,1,SHT,T,4,合作,1
2,3935430746,2017-06-20 15:03:39,366,30,3403530,NaN,18594989,F,2017-07-01,2017-07-04,12,2017-06-20 15:03:53,S,1118.1,89.71200,0,33727477.0,741340,3686.0,1,SHT,F,5,合作,1
3,4095204178,2017-07-19 23:45:12,274,42,4984070,705.0,72423477,T,2017-08-22,2017-08-24,34,2017-07-19 23:45:14,S,250.5,22.45260,0,52394065.0,5033399,2299.0,0,HTL,T,5,直签,1
4,3753007353,2017-05-26 16:21:36,723,108,3796626,340.0,55656893,F,2017-05-27,2017-05-28,1,NaN,C,192.3,17.44395,0,6132999.0,2144683,2391.0,0,SHT,T,5,合作,1


(11035, 19)

(11036, 19)

,orderid,orderdate,city,countryid,hotel,zone,room,isholdroom,arrival,etd,ordadvanceday,masterbasicroomid,masterhotelid,supplierid,isvendor,hotelbelongto,isebookinghtl,hotelstar,supplierchannel
0,4505896718,2017/9/15 14:11,633,66,6265739,-1.0,71340604,T,2017/10/11,2017/10/15,27,22012743,5633633,2299,0,HTL,T,3,直签
1,4504466510,2017/9/15 4:32,410,42,2811117,NaN,70578904,F,2017/9/19,2017/9/20,4,24779139,688942,3536,1,SHT,F,3,合作
2,4505791849,2017/9/15 13:43,301,111,8198646,853.0,91096440,F,2017/9/16,2017/9/17,2,50438281,7071803,2322,1,SHT,F,3,合作
3,4505284465,2017/9/15 11:38,274,42,3717320,705.0,20487676,F,2017/9/17,2017/9/20,2,11745244,991852,3965,1,SHT,F,0,合作
4,4506807614,2017/9/15 17:38,219,78,7217820,-1.0,88163053,T,2017/9/17,2017/9/20,2,38411936,7217821,2299,0,HTL,T,4,直签


In [4]:
test = pd.concat([testa, testb])
del testa
del testb
train = train[list(test.columns)]
df = pd.concat([train, test])
df.reset_index(drop=True, inplace=True)
del train
del test
gc.collect()

df['orderdate'] = pd.to_datetime(df['orderdate'], format='%Y-%m-%d %H:%M')
df['arrival'] = pd.to_datetime(df['arrival'], format='%Y-%m-%d %H:%M')
df['etd'] = pd.to_datetime(df['etd'], format='%Y-%m-%d %H:%M')

df['orderdate_mon'] = df['orderdate'].apply(lambda x:x.month)
df['arrival_mon'] = df['arrival'].apply(lambda x:x.month)
df['etd_mon'] = df['etd'].apply(lambda x:x.month)

df['orderdate_day'] = df['orderdate'].apply(lambda x:x.day)
df['arrival_day'] = df['arrival'].apply(lambda x:x.day)
df['etd_day'] = df['etd'].apply(lambda x:x.day)

47

In [5]:
df.shape
# df.isnull().sum()

(4045157, 25)

## 分表查看

In [6]:
zqroom = pd.read_csv('./data/train/ord_zqroomstatus.csv')
bkroom = pd.read_csv('./data/train/ord_bkroomstatus.csv')
zhotel = pd.read_csv('./data/train/hotelinfo.csv')
mhotel = pd.read_csv('./data/train/mhotelinfo.csv')
mroom = pd.read_csv('./data/train/mroominfo.csv')
chaifen = pd.read_csv('./data/train/ord_chaifen.csv')

zqroom.shape
bkroom.shape
zhotel.shape
mhotel.shape
mroom.shape
chaifen.shape

(2209315, 7)

(158857, 7)

(4629010, 6)

(1487009, 7)

(21898126, 3)

(9050289, 3)

In [7]:
zqroom.head()
bkroom.head()
zhotel.head()
mhotel.head()
mroom.head()
chaifen.head()

,room,arrival,roomstatus,createtime,masterbasicroomid,hotel,masterhotelid
0,67304804,2017-09-09,N,2017-08-25 15:10:14,5179572,5240872,1687209
1,48070785,2017-08-30,G,2017-08-20 10:38:03,5980276,4517763,1761540
2,27442833,2017-10-23,N,2017-09-14 11:57:28,5733411,2714517,1005498
3,85683868,2017-10-23,G,2017-09-07 20:18:25,18075025,3752555,1337909
4,70172876,2017-10-07,L,2017-08-07 16:15:09,10907169,7592369,706631


,room,arrival,roomstatus,createtime,masterbasicroomid,hotel,masterhotelid
0,49359351,2017-10-01,N,2017-08-24 03:54:17,44039493,4076704,1857089
1,13037426,2017-08-25,N,2017-08-05 14:34:38,7924733,229256,699276
2,8573004,2017-08-10,N,2017-08-07 18:25:40,7769214,538682,688203
3,13619122,2017-08-22,N,2017-08-05 09:44:34,5787108,785102,1074098
4,39616190,2017-08-13,N,2017-08-05 19:15:17,18215309,5114310,5086203


,hotel,totalrooms,isebookinghtl,hotelbelongto,supplierid,masterhotelid
0,8164,2,F,PKG,NaN,8164
1,19660,4,F,HTL,NaN,19660
2,21524,2,F,HTL,NaN,21524
3,21653,1,F,HTL,2654.0,994188
4,22486,2,F,HTL,NaN,22486


,masterhotelid,glon,glat,star,zone,city,country
0,8657540,151.346340,-33.516495,0,NaN,4129,15
1,10095686,92.526480,58.190506,0,NaN,71743,30
2,15935739,34.558693,49.588090,0,NaN,5615,119
3,15156172,110.363140,-7.771237,2,0.0,741,108
4,10950378,20.406840,44.806694,0,2298.0,10257,183


,masterbasicroomid,totalrooms,masterhotelid
0,10023629,1,3177583
1,10030510,3,2193611
2,10031578,2,735007
3,10032516,2,3035324
4,10034229,588,757098


,orderid,arrival,etd
0,3038243565,2017-02-12,2017-02-13
1,3053417848,2017-04-21,2017-04-24
2,3061249423,2017-03-03,2017-03-04
3,3075780075,2017-02-10,2017-02-12
4,3075944254,2017-02-25,2017-03-01


### chaifen

In [22]:
chaifen.head()

,orderid,arrival,etd
0,3038243565,2017-02-12,2017-02-13
1,3053417848,2017-04-21,2017-04-24
2,3061249423,2017-03-03,2017-03-04
3,3075780075,2017-02-10,2017-02-12
4,3075944254,2017-02-25,2017-03-01


In [75]:
df.head()

,orderid,orderdate,city,countryid,hotel,zone,room,isholdroom,arrival,etd,ordadvanceday,masterbasicroomid,masterhotelid,supplierid,isvendor,hotelbelongto,isebookinghtl,hotelstar,supplierchannel,orderdate_mon,arrival_mon,etd_mon,orderdate_day,arrival_day,etd_day,order_day
0,3678052984,2017-05-15 10:03:34,1777,111,6459206,3302.0,62786796,F,2017-06-23,2017-06-25,40,21087625.0,713478,1897.0,0,SHT,T,5,合作,5,6,6,15,23,25,2
1,3623712471,2017-05-02 21:25:37,622,4,4515821,NaN,51183791,F,2017-05-05,2017-05-06,3,30110898.0,1210465,2322.0,1,SHT,T,4,合作,5,5,5,2,5,6,1
2,3935430746,2017-06-20 15:03:39,366,30,3403530,NaN,18594989,F,2017-07-01,2017-07-04,12,33727477.0,741340,3686.0,1,SHT,F,5,合作,6,7,7,20,1,4,3
3,4095204178,2017-07-19 23:45:12,274,42,4984070,705.0,72423477,T,2017-08-22,2017-08-24,34,52394065.0,5033399,2299.0,0,HTL,T,5,直签,7,8,8,19,22,24,2
4,3753007353,2017-05-26 16:21:36,723,108,3796626,340.0,55656893,F,2017-05-27,2017-05-28,1,6132999.0,2144683,2391.0,0,SHT,T,5,合作,5,5,5,26,27,28,1


In [18]:
tmp = chaifen.groupby(['orderid'], as_index=False).count()
tmp.shape

(4045157, 3)

In [8]:
%%time
def get_order_day(x):
    return (x.etd - x.arrival).days    

df['order_day'] = df.apply(get_order_day, axis=1)

In [19]:
order = pd.DataFrame()
order['orderid'] = chaifen.orderid.unique()
order = order.merge(tmp[['orderid', 'arrival']], on=['orderid'], how='left')
order = order.merge(df[['orderid', 'order_day']], on=['orderid'], how='left')
order.columns = ['orderid', 'actual_day', 'order_day']

order.shape
order.head()

(4045157, 3)

,orderid,actual_day,order_day
0,3038243565,10,10
1,3053417848,4,4
2,3061249423,3,3
3,3075780075,2,2
4,3075944254,4,4


In [20]:
%%time
def is_equal(x):
    if x.actual_day == x.order_day:
        return 1
    else:
        return 0

order['is_equal'] = order.apply(is_equal, axis=1)

In [21]:
order.groupby(['is_equal']).orderid.count()

is_equal
0       1909
1    4043248
Name: orderid, dtype: int64

In [61]:
order[(order.is_equal == 0) & (order.actual_day == 0)].shape

(1905, 4)

In [62]:
order[(order.is_equal == 0) & (order.actual_day != 0)]

,orderid,actual_day,order_day,is_equal
287078,4003340216,4,5,0
1437608,4337362450,1,5,0
2984485,3916415821,1,2,0
3747056,4441319557,2,3,0


In [79]:
# order.to_csv('./features/order.csv', index=False)

### zqroom/bkroom

In [25]:
zqroom.head(2) # 部分关房
bkroom.head(2) # 全关房

,room,arrival,roomstatus,createtime,masterbasicroomid,hotel,masterhotelid
0,67304804,2017-09-09,N,2017-08-25 15:10:14,5179572,5240872,1687209
1,48070785,2017-08-30,G,2017-08-20 10:38:03,5980276,4517763,1761540


,room,arrival,roomstatus,createtime,masterbasicroomid,hotel,masterhotelid
0,49359351,2017-10-01,N,2017-08-24 03:54:17,44039493,4076704,1857089
1,13037426,2017-08-25,N,2017-08-05 14:34:38,7924733,229256,699276


In [69]:
zqroom.shape[0]
len(zqroom.room.unique())
bkroom.shape[0]
len(bkroom.room.unique())
len(set(zqroom.room.unique()) & (set(bkroom.room.unique())))
len(df.room.unique())
len(set(list(zqroom.room.unique()) + (list(bkroom.room.unique()))))

2209315

280610

158857

7628

6055

766701

282183

In [31]:
zqroom[zqroom.room == 61964290]
bkroom[bkroom.room == 61964290]

,room,arrival,roomstatus,createtime,masterbasicroomid,hotel,masterhotelid
283031,61964290,2017-08-13,N,2017-08-08 20:34:09,34697820,7210941,7624218


,room,arrival,roomstatus,createtime,masterbasicroomid,hotel,masterhotelid
17922,61964290,2017-08-13,N,2017-08-12 07:38:25,34697820,7210941,7624218
99590,61964290,2017-08-13,N,2017-08-17 07:52:48,34697820,7210941,7624218
100723,61964290,2017-08-12,N,2017-08-17 05:26:52,34697820,7210941,7624218
112616,61964290,2017-08-12,N,2017-08-11 12:22:06,34697820,7210941,7624218
146215,61964290,2017-08-13,N,2017-08-13 09:24:02,34697820,7210941,7624218


### zhotel/mhotel

In [32]:
zhotel.head(2)
mhotel.head(2)

,hotel,totalrooms,isebookinghtl,hotelbelongto,supplierid,masterhotelid
0,8164,2,F,PKG,NaN,8164
1,19660,4,F,HTL,NaN,19660


,masterhotelid,glon,glat,star,zone,city,country
0,8657540,151.34634,-33.516495,0,NaN,4129,15
1,10095686,92.52648,58.190506,0,NaN,71743,30


In [37]:
zhotel.shape[0]
len(zhotel.hotel.unique())
mhotel.shape[0]
len(mhotel.masterhotelid.unique())

4629010

4629010

1487009

1487009

### mroom

In [34]:
mroom.head()

,masterbasicroomid,totalrooms,masterhotelid
0,10023629,1,3177583
1,10030510,3,2193611
2,10031578,2,735007
3,10032516,2,3035324
4,10034229,588,757098


In [51]:
mroom.shape[0]
len(mroom.masterbasicroomid.unique())

21898126

21898029

In [84]:
# tmp = mroom.groupby(['masterbasicroomid']).masterbasicroomid.count()
tmp[tmp != 1].index

Int64Index([ 2176208,  2176209,  2176210,  2969827,  4189515,  4253268,
             4254089,  4263972,  4263974,  4327510,  5025576,  6107948,
             6465521,  6684078,  6934420,  6934444, 10038483, 10038542,
            10066605, 10073721, 10073733, 10239875, 10255456, 10543893,
            10543894, 10543895, 14044414, 14577978, 17146213, 18169193,
            18169197, 18237230, 18381434, 18381438, 18872013, 18972077,
            19031243, 19031246, 19215504, 20582141, 20687952, 21283447,
            21887171, 24021902, 26119285, 26182050, 28778421, 29580265,
            29837173, 32175413, 32350105, 35414556, 39409103, 39423655,
            39781470, 39781488, 39784926, 39786751, 39786765, 39786784,
            39786801, 39786816, 39838163, 39845023, 39967254, 39967265,
            39967275, 39979230, 40115723, 40115729, 40115755, 40137597,
            40137619, 40155806, 40155816, 40159106, 41120990, 46628383,
            46628440, 46628751, 46628954, 52637690, 53805192, 55

In [85]:
mroom[mroom.masterbasicroomid == 40115723]

,masterbasicroomid,totalrooms,masterhotelid
16886428,40115723,3,3799195
17144411,40115723,3,2914657
